# DNBLab Jupyter Notebook zum Datenbezug über die SRU-Schnittstelle

Dieses DNBLab-Tutorial beschreibt eine Beispielabfrage über die SRU-Schnittstelle mit Python. In der Jupyter Notebook Umgebung kann der dokumentierte Code direkt ausgeführt und angepasst werden. Das Tutorial umfasst die exemplarische Abfrage und Ausgabe der Daten in MARC21-xml zur weiteren Verarbeitung mit Python. 

## Arbeitsumgebung einrichten <a class="anchor" id="Teil1"></a>

In [1]:
import urllib.parse
from pyodide.http import open_url, pyfetch
from js import fetch
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd

## Abfrage über alle Datensätze <a class="anchor" id="Teil2"></a>

In [2]:
async def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    
    r = await fetch(base_url + "?" + urllib.parse.urlencode(params))  
    r_text = await r.text()
    xml = soup(r_text, features="xml")
    records = xml.find_all('record', {'type':'Bibliographic'})
    
    
    if len(records) < 100:
        
        return records
    
    else:
        
        num_results = 100
        i = 101
        while num_results == 100:
            
            params.update({'startRecord': i})
            r = await fetch(base_url + "?" + urllib.parse.urlencode(params))  
            r_text = await r.text()
            xml = soup(r_text, features="xml")
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records

## Felder Titel und Links zu den Objekten durchsuchen <a class="anchor" id="Teil3"></a>

In [4]:
def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
    
    # link
    link = xml.xpath("marc:datafield[@tag = '856']/marc:subfield[@code = 'u']", namespaces=ns)
    
    try:
        link = link[0].text
        #titel = unicodedata.normalize("NFC", titel)
    except:
        link = "unkown"
        
          # titel
    titel = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    
    try:
        titel = titel[0].text
        #titel = unicodedata.normalize("NFC", titel)
    except:
        titel = "unkown"
        
    
    meta_dict = {"idn":idn,
                 "titel":titel,
                 "link":link}
    
    return meta_dict

## Abfrage Projekt 28 (cod=d028)

In [9]:
records = await dnb_sru('cod=d028')
print(len(records), 'Ergebnisse')

196 Ergebnisse


## Anzeige der Treffer in einer Tabelle  <a class="anchor" id="Teil4"></a>

In [10]:
output = [parse_record(record) for record in records]
df = pd.DataFrame(output)
df

,idn,titel,link
0,1163616249,13 Konkrete,https://nbn-resolving.org/urn:nbn:de:101:1-201...
1,1163613266,200 Jahre Frommann-Verlag,https://nbn-resolving.org/urn:nbn:de:101:1-201...
2,1163614645,60 Fotos,https://nbn-resolving.org/urn:nbn:de:101:1-201...
3,1163615773,Abendgesang (Neila),https://nbn-resolving.org/urn:nbn:de:101:1-201...
4,1163615218,Abstraktion in der Malerei,https://nbn-resolving.org/urn:nbn:de:101:1-201...
...,...,...,...
191,1163612235,Willy Baumeister,https://nbn-resolving.org/urn:nbn:de:101:1-201...
192,1163615420,Zeitberichte und Dokumente,https://nbn-resolving.org/urn:nbn:de:101:1-201...
193,1163614211,Ziele des Schriftunterrichts,https://nbn-resolving.org/urn:nbn:de:101:1-201...
194,1163614912,Zum neuen Bauen,https://nbn-resolving.org/urn:nbn:de:101:1-201...


# Speichern der Ergebnisse als CSV-Datei!

In [8]:
df.to_csv("SRU_Titel.csv", index=False)